# Wrangling Exercises

In [1]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split

import pip

## Acquire and Summarize

### 1. Acquire data from the cloud database.

In [2]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
sql_query = '''
    SELECT 
    *
    FROM properties_2017
    RIGHT JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
    LEFT JOIN propertylandusetype USING (propertylandusetypeid)
    LEFT JOIN airconditioningtype USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
    LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
    WHERE propertylandusedesc = "Single Family Residential"
    AND predictions_2017.transactiondate LIKE "2017%%"
    AND latitude IS NOT NULL
    AND longitude IS NOT NULL;
'''
    
def get_zillow_data():
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df

In [3]:
df = get_zillow_data()

In [4]:
df.shape

(52441, 69)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   heatingorsystemtypeid         33935 non-null  float64
 2   buildingclasstypeid           0 non-null      object 
 3   architecturalstyletypeid      70 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   propertylandusetypeid         52441 non-null  float64
 6   id                            52441 non-null  int64  
 7   parcelid                      52441 non-null  int64  
 8   logerror                      52441 non-null  float64
 9   transactiondate               52441 non-null  object 
 10  id                            52441 non-null  int64  
 11  parcelid                      52441 non-null  int64  
 12  basementsqft                  47 non-null     float64
 13  b

### 2. Summarize your data

In [6]:
df.taxamount.dtype

dtype('float64')

In [7]:
for col in df.columns:
    if col != 'id':
        if col != 'parcelid':
            print(col)
            print(df[col].value_counts())

typeconstructiontypeid
6.00    75
4.00     1
Name: typeconstructiontypeid, dtype: int64
heatingorsystemtypeid
 2.00    20736
 7.00    12565
 6.00      517
20.00       85
13.00       16
 1.00        7
18.00        6
10.00        2
24.00        1
Name: heatingorsystemtypeid, dtype: int64
buildingclasstypeid
Series([], Name: buildingclasstypeid, dtype: int64)
architecturalstyletypeid
 7.00    62
 3.00     3
 2.00     2
21.00     2
 8.00     1
Name: architecturalstyletypeid, dtype: int64
airconditioningtypeid
 1.00    11895
13.00     1568
 5.00      159
11.00       16
Name: airconditioningtypeid, dtype: int64
propertylandusetypeid
261.00    52441
Name: propertylandusetypeid, dtype: int64
logerror
 0.00    5
 0.00    5
 0.00    5
 0.00    5
 0.02    4
        ..
 0.02    1
 0.02    1
-0.03    1
 0.03    1
 0.04    1
Name: logerror, Length: 52180, dtype: int64
transactiondate
2017-06-30    810
2017-04-28    617
2017-05-31    566
2017-07-28    546
2017-08-31    528
             ... 
2017-05-0

Name: unitcnt, dtype: int64
yardbuildingsqft17
  200.00    85
  240.00    71
  300.00    49
  400.00    37
  288.00    34
            ..
  319.00     1
1,113.00     1
  415.00     1
  913.00     1
  201.00     1
Name: yardbuildingsqft17, Length: 504, dtype: int64
yardbuildingsqft26
120.00    3
 56.00    2
300.00    2
240.00    2
448.00    2
196.00    1
140.00    1
 48.00    1
284.00    1
 39.00    1
133.00    1
486.00    1
250.00    1
 91.00    1
491.00    1
 16.00    1
204.00    1
 71.00    1
 28.00    1
195.00    1
292.00    1
231.00    1
 30.00    1
320.00    1
384.00    1
180.00    1
 49.00    1
868.00    1
462.00    1
461.00    1
368.00    1
105.00    1
 37.00    1
 88.00    1
160.00    1
 45.00    1
752.00    1
169.00    1
 21.00    1
200.00    1
113.00    1
 41.00    1
314.00    1
 12.00    1
414.00    1
 66.00    1
648.00    1
174.00    1
218.00    1
 60.00    1
336.00    1
108.00    1
144.00    1
216.00    1
360.00    1
 72.00    1
576.00    1
Name: yardbuildingsqft26, dtype: 

In [8]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [9]:
#summarize(df)

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.

In [10]:
# Counting attribute nulls (via column)
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [11]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclasstypeid,52441,100.00
buildingclassdesc,52441,100.00
finishedsquarefeet15,52441,100.00
finishedsquarefeet13,52441,100.00
basementsqft,52394,99.91
...,...,...
logerror,0,0.00
fips,0,0.00
latitude,0,0.00
parcelid,0,0.00


## Prepare

In [12]:
# for now, i'll use the trim_data function to prep my data
def trim_bad_data_zillow(df):
    # If it's not single unit, it's not a single family home.
    df = df[~(df.unitcnt > 1)]
    # If the lot size is smaller than the finished square feet, it's probably bad data or not a single family home
    df = df[~(df.lotsizesquarefeet < df.calculatedfinishedsquarefeet)]
    # If the finished square feet is less than 500 it is likeley an apartment, or bad data
    df = df[~(df.calculatedfinishedsquarefeet < 500)]
    # If there are no bedrooms, likely a loft or bad data
    df = df[~(df.bedroomcnt < 1)]
    # Drop duplicate parcels
    df = df.drop_duplicates(subset='parcelid')
    return df

In [13]:
# df = trim_bad_data_zillow(df)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   heatingorsystemtypeid         33935 non-null  float64
 2   buildingclasstypeid           0 non-null      object 
 3   architecturalstyletypeid      70 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   propertylandusetypeid         52441 non-null  float64
 6   id                            52441 non-null  int64  
 7   parcelid                      52441 non-null  int64  
 8   logerror                      52441 non-null  float64
 9   transactiondate               52441 non-null  object 
 10  id                            52441 non-null  int64  
 11  parcelid                      52441 non-null  int64  
 12  basementsqft                  47 non-null     float64
 13  b

In [15]:
df['fullbathcnt'].value_counts()

 2.00    25831
 3.00    11600
 1.00    10402
 4.00     2915
 5.00     1022
 6.00      359
 7.00       96
 8.00       56
 9.00       13
10.00        5
11.00        3
18.00        1
13.00        1
Name: fullbathcnt, dtype: int64

In [16]:
df = df.drop(columns = ['typeconstructiontypeid','heatingorsystemtypeid','buildingclasstypeid','architecturalstyletypeid','airconditioningtypeid','propertylandusetypeid','basementsqft','decktypeid','finishedfloor1squarefeet','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','fireplacecnt','hashottuborspa','poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7','storytypeid','yardbuildingsqft17','yardbuildingsqft26','numberofstories','fireplaceflag','taxdelinquencyflag','taxdelinquencyyear','architecturalstyledesc','buildingclassdesc','typeconstructiondesc'],axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52441 non-null  int64  
 1   parcelid                      52441 non-null  int64  
 2   logerror                      52441 non-null  float64
 3   transactiondate               52441 non-null  object 
 4   id                            52441 non-null  int64  
 5   parcelid                      52441 non-null  int64  
 6   bathroomcnt                   52441 non-null  float64
 7   bedroomcnt                    52441 non-null  float64
 8   buildingqualitytypeid         33740 non-null  float64
 9   calculatedbathnbr             52304 non-null  float64
 10  calculatedfinishedsquarefeet  52359 non-null  float64
 11  finishedsquarefeet12          52194 non-null  float64
 12  fips                          52441 non-null  float64
 13  f

In [18]:
df['buildingqualitytypeid'].value_counts()

 6.00    10289
 8.00     8244
 4.00     8156
 7.00     3427
 5.00     1513
 9.00     1121
11.00      514
10.00      230
 3.00      152
12.00       81
 1.00        8
 2.00        5
Name: buildingqualitytypeid, dtype: int64

In [19]:
df['censustractandblock'].value_counts()

60,379,201,162,006.00    32
60,379,203,391,054.00    27
60,590,320,571,003.00    24
60,371,417,002,000.00    24
60,590,320,421,002.00    23
                         ..
60,372,714,001,001.00     1
60,371,954,004,007.00     1
60,379,108,051,066.00     1
60,590,758,081,024.00     1
60,375,034,021,020.00     1
Name: censustractandblock, Length: 31404, dtype: int64

In [20]:
df

,id,parcelid,logerror,transactiondate,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,propertylandusedesc,airconditioningdesc,heatingorsystemdesc
0,0,14297519,0.03,2017-01-01,1727539,14297519,3.50,4.00,NaN,3.50,...,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",Single Family Residential,None,None
1,1,17052889,0.06,2017-01-01,1387261,17052889,1.00,2.00,NaN,1.00,...,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",Single Family Residential,None,None
2,2,14186244,0.01,2017-01-01,11677,14186244,2.00,3.00,NaN,2.00,...,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",Single Family Residential,None,None
3,3,12177905,-0.10,2017-01-01,2288172,12177905,3.00,4.00,8.00,3.00,...,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",Single Family Residential,None,Central
4,6,12095076,-0.00,2017-01-01,781532,12095076,3.00,4.00,9.00,3.00,...,"1,950.00","276,684.00","773,303.00","2,016.00","496,619.00","9,516.26","60,374,608,001,014.00",Single Family Residential,Central,Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,77607,12412492,0.00,2017-09-19,2274245,12412492,2.00,4.00,6.00,2.00,...,"1,962.00","125,466.00","346,534.00","2,016.00","221,068.00","4,175.08","60,375,548,022,001.00",Single Family Residential,None,Central
52437,77609,11000655,0.02,2017-09-20,673515,11000655,2.00,2.00,6.00,2.00,...,"1,940.00","70,917.00","354,621.00","2,016.00","283,704.00","4,478.43","60,371,014,003,002.00",Single Family Residential,None,Central
52438,77610,17239384,0.01,2017-09-21,2968375,17239384,2.00,4.00,NaN,2.00,...,"1,964.00","50,683.00","67,205.00","2,016.00","16,522.00","1,107.48","61,110,084,022,016.00",Single Family Residential,None,None
52439,77611,12773139,0.04,2017-09-21,1843709,12773139,1.00,3.00,4.00,1.00,...,"1,954.00","32,797.00","49,546.00","2,016.00","16,749.00",876.43,"60,374,338,022,005.00",Single Family Residential,Central,Central


In [21]:
df['samebath'] = np.where(df['bathroomcnt'] == df['calculatedbathnbr'], 1, 0)
df['samebath'].value_counts()

1    52304
0      137
Name: samebath, dtype: int64

In [22]:
df['logerror'].mean()

0.018298494294413003

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52441 non-null  int64  
 1   parcelid                      52441 non-null  int64  
 2   logerror                      52441 non-null  float64
 3   transactiondate               52441 non-null  object 
 4   id                            52441 non-null  int64  
 5   parcelid                      52441 non-null  int64  
 6   bathroomcnt                   52441 non-null  float64
 7   bedroomcnt                    52441 non-null  float64
 8   buildingqualitytypeid         33740 non-null  float64
 9   calculatedbathnbr             52304 non-null  float64
 10  calculatedfinishedsquarefeet  52359 non-null  float64
 11  finishedsquarefeet12          52194 non-null  float64
 12  fips                          52441 non-null  float64
 13  f

In [24]:
df.isnull().sum()

id                                  0
parcelid                            0
logerror                            0
transactiondate                     0
id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18701
calculatedbathnbr                 137
calculatedfinishedsquarefeet       82
finishedsquarefeet12              247
fips                                0
fullbathcnt                       137
garagecarcnt                    34426
garagetotalsqft                 34426
latitude                            0
longitude                           0
lotsizesquarefeet                 369
poolcnt                         41345
propertycountylandusecode           0
propertyzoningdesc              18593
rawcensustractandblock              0
regionidcity                     1037
regionidcounty                      0
regionidneighborhood            33408
regionidzip 

In [25]:
# What's the percentage of nulls?
df.isna().mean()

id                                             0.00
parcelid                                       0.00
logerror                                       0.00
transactiondate                                0.00
id                                             0.00
parcelid                                       0.00
bathroomcnt                                    0.00
bedroomcnt                                     0.00
buildingqualitytypeid                          0.36
calculatedbathnbr                              0.00
calculatedfinishedsquarefeet                   0.00
finishedsquarefeet12                           0.00
fips                                           0.00
fullbathcnt                                    0.00
garagecarcnt                                   0.66
garagetotalsqft                                0.66
latitude                                       0.00
longitude                                      0.00
lotsizesquarefeet                              0.01
poolcnt     

In [26]:
df['poolcnt'].value_counts()

1.00    11096
Name: poolcnt, dtype: int64

In [27]:
df['poolcnt']

0                        NaN
1                        NaN
2                       1.00
3                       1.00
4                       1.00
                ...         
52436                    NaN
52437                    NaN
52438                    NaN
52439                    NaN
52440                    NaN
Name: poolcnt, Length: 52441, dtype: float64

In [28]:
# NaN values in poolcnt most likely indicates no pool, thus I convert Nan vlaues to '0'
df['poolcnt'] = df['poolcnt'].fillna(0)

In [29]:
df['garagecarcnt'].value_counts()

 2.00    14802
 1.00     2202
 3.00      621
 4.00      254
 0.00       65
 5.00       45
 6.00       13
 7.00        3
10.00        3
 8.00        2
 9.00        2
11.00        1
13.00        1
14.00        1
Name: garagecarcnt, dtype: int64

In [30]:
df['garagecarcnt'].isna().value_counts()

True     34426
False    18015
Name: garagecarcnt, dtype: int64

In [31]:
# NaN values in garagecarcnt most likely indicates no garage, thus I convert Nan vlaues to '0'
df['garagecarcnt'] = df['garagecarcnt'].fillna(0)

In [32]:
df['censustractandblock'].value_counts()

60,379,201,162,006.00    32
60,379,203,391,054.00    27
60,590,320,571,003.00    24
60,371,417,002,000.00    24
60,590,320,421,002.00    23
                         ..
60,372,714,001,001.00     1
60,371,954,004,007.00     1
60,379,108,051,066.00     1
60,590,758,081,024.00     1
60,375,034,021,020.00     1
Name: censustractandblock, Length: 31404, dtype: int64

In [33]:
df['raw_vs_reg'] = np.where(df['rawcensustractandblock'] == df['censustractandblock'], 1, 0)
df['raw_vs_reg'].value_counts()

0    52441
Name: raw_vs_reg, dtype: int64

In [34]:
df['rawcensustractandblock']

0              60,590,630.07
1              61,110,010.02
2              60,590,218.02
3              60,373,001.00
4              60,374,608.00
                ...         
52436          60,375,548.02
52437          60,371,014.00
52438          61,110,084.02
52439          60,374,338.02
52440          60,375,034.02
Name: rawcensustractandblock, Length: 52441, dtype: float64

In [35]:
df['censustractandblock']

0       60,590,630,072,012.00
1       61,110,010,023,006.00
2       60,590,218,022,012.00
3       60,373,001,001,006.00
4       60,374,608,001,014.00
                 ...         
52436   60,375,548,022,001.00
52437   60,371,014,003,002.00
52438   61,110,084,022,016.00
52439   60,374,338,022,005.00
52440   60,375,034,021,020.00
Name: censustractandblock, Length: 52441, dtype: float64

In [36]:
df['lotsizesquarefeet'].mean()

11339.624078199417

In [38]:
df['lotsizesquarefeet'].isna().sum()

369

In [45]:
df['lotsizesquarefeet'] = df['lotsizesquarefeet'].fillna(value=df['lotsizesquarefeet'].mean())

In [46]:
df['lotsizesquarefeet'].isna().sum()

0

In [48]:
df.isna().mean()

id                                             0.00
parcelid                                       0.00
logerror                                       0.00
transactiondate                                0.00
id                                             0.00
parcelid                                       0.00
bathroomcnt                                    0.00
bedroomcnt                                     0.00
buildingqualitytypeid                          0.36
calculatedbathnbr                              0.00
calculatedfinishedsquarefeet                   0.00
finishedsquarefeet12                           0.00
fips                                           0.00
fullbathcnt                                    0.00
garagecarcnt                                   0.00
garagetotalsqft                                0.66
latitude                                       0.00
longitude                                      0.00
lotsizesquarefeet                              0.00
poolcnt     

In [60]:
df['unitcnt'].value_counts()

1.00    33817
2.00       29
3.00        1
Name: unitcnt, dtype: int64

In [63]:
df['unitcnt'].isna().value_counts()

False    33847
True     18594
Name: unitcnt, dtype: int64

In [67]:
df.isna().mean()

id                                             0.00
parcelid                                       0.00
logerror                                       0.00
transactiondate                                0.00
id                                             0.00
parcelid                                       0.00
bathroomcnt                                    0.00
bedroomcnt                                     0.00
buildingqualitytypeid                          0.36
calculatedbathnbr                              0.00
calculatedfinishedsquarefeet                   0.00
finishedsquarefeet12                           0.00
fips                                           0.00
fullbathcnt                                    0.00
garagecarcnt                                   0.00
garagetotalsqft                                0.66
latitude                                       0.00
longitude                                      0.00
lotsizesquarefeet                              0.00
poolcnt     

In [69]:
df.dropna().shape

(0, 42)

In [68]:
# If we drop all rows containing any null, how much data do we lose?
round(df.dropna().shape[0] / df.shape[0], 4)

0.0

In [ ]:
df.dropna()

In [ ]:
df.head()

In [ ]:
# Potential function for removing properties other than single unit properties
def trim_bad_data_zillow(df):
    # If it's not single unit, it's not a single family home.
    df = df[~(df.unitcnt > 1)]
    # If the lot size is smaller than the finished square feet, it's probably bad data or not a single family home
    df = df[~(df.lotsizesquarefeet < df.calculatedfinishedsquarefeet)]
    # If the finished square feet is less than 500 it is likeley an apartment, or bad data
    df = df[~(df.calculatedfinishedsquarefeet < 500)]
    # If there are no bedrooms, likely a loft or bad data
    df = df[~(df.bedroomcnt < 1)]
    # Drop duplicate parcels
    df = df.drop_duplicates(subset='parcelid')
    return df


#Dropping columns with an inordinate number of nulls (rendering variable essentially useless)
def drop_bad_columns(df):
    df = df.drop(columns = [
        'typeconstructiontypeid',
        'heatingorsystemtypeid',
        'buildingclasstypeid',
        'architecturalstyletypeid',
        'airconditioningtypeid',
        'propertylandusetypeid',
        'basementsqft',
        'decktypeid',
        'finishedfloor1squarefeet',
        'finishedsquarefeet13',
        'finishedsquarefeet15',
        'finishedsquarefeet50',
        'finishedsquarefeet6',
        'fireplacecnt',
        'hashottuborspa',
        'poolsizesum',
        'pooltypeid10',
        'pooltypeid2',
        'pooltypeid7',
        'storytypeid',
        'yardbuildingsqft17',
        'yardbuildingsqft26',
        'numberofstories',
        'fireplaceflag',
        'taxdelinquencyflag',
        'taxdelinquencyyear',
        'architecturalstyledesc',
        'buildingclassdesc',
        'typeconstructiondesc',
        'buildingqualitytypeid',
        'propertyzoningdesc',
        'rawcensustractandblock',
        'regionidneighborhood',
        'threequarterbathnbr',
        'airconditioningdesc',
        'heatingorsystemdesc',
        'threequarterbathnbr'
        ],
        axis=1)

    return df
    
def drop_nulls(df):
    # Change all remaining null/nan values to 0 or the variable mean, depending on best use case
    df['poolcnt'] = df['poolcnt'].fillna(0)
    df['garagecarcnt'] = df['garagecarcnt'].fillna(0)
    df['garagetotalsqft'] = df['garagetotalsqft'].fillna(0)
    df['lotsizesquarefeet'] = df['lotsizesquarefeet'].fillna(value=df['lotsizesquarefeet'].mean())
    df['regionidcity'] = df['regionidcity'].fillna(0)
    df['unitcnt'] = df['unitcnt'].fillna(0)

    return df

def handle_nulls(df):    
    # We keep 99.41% of the data after dropping nulls
    # round(df.dropna().shape[0] / df.shape[0], 4) returned .9941
    df = df.dropna()
    return df


def optimize_types(df):
    # Convert some columns to integers for optimization
    # fips, yearbuilt, and bedrooms, taxvaluedollarcnt, and calculatedfinishedsquarefeet can be integers
    df["fips"] = df["fips"].astype(int)
    df["yearbuilt"] = df["yearbuilt"].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)    
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    return df


def handle_outliers(df):
    """Manually handle outliers that do not represent properties likely for 99% of buyers and zillow visitors"""
    df = df[df.bathroomcnt <= 6]
    
    df = df[df.bedroomcnt <= 6]

    df = df[df.taxvaluedollarcnt < 1_500_000]

    return df

def clean_variables(df):
    # Drop 'taxamount' column (variable is inconsistent based on time and location of collected value, could lead to poor analysis)
    # Rename columns and 'fips' values to reflect actual location (to solidify column as categorical variable)
    df = df.drop(columns = 'taxamount')
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                              'bathroomcnt':'bathrooms', 
                              'calculatedfinishedsquarefeet':'sq_ft', 
                              'taxvaluedollarcnt':'home_value', 
                              'yearbuilt':'year_built', 
                              'fips':'location',
                              'fullbathcnt':'full_bathrooms',
                              'garagecarcnt':'garage_spaces',
                              'lotsizesquarefeet':'lot_sq_ft'
                             })
    df.location = df.location.replace(to_replace={6037:'LA County', 6059:'Orange County', 6111:'Ventura County'})

    return df 

def feature_engineering(df):
    # Bin `year_built` by decade
    df["decade_built"] = pd.cut(x=df["year_built"], bins=[1800, 1899, 1909, 1919, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999, 2009], labels=['1800s', '1900s', '10s', '20s', '30s', '40s', '50s', '60s', '70s', '80s', '90s', '2000s'])
    # Convert categorical variable to numeric var
    df['county_encoded'] = df.location.map({'LA County': 0, 'Orange County': 1, 'Ventura County': 2})

    df = df.dropna()

    return df

# Split for Exploration

## 
# Train, Validate, Test Split Function: for exploration
def zillow_split_explore(df):
    '''
    This function performs split on telco data, stratifying on churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2,
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3,
                                   random_state=123)
    return train, validate, test

### ------------------------------------------------------------------------

# Split for Modeling: X & Y dfs
def zillow_split_model(df):
    '''
    This function performs split on telco data, stratifying on churn.
    Returns both X and y train, validate, and test dfs
    '''
    
    train_validate, test = train_test_split(df, test_size=.2,
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3,
                                   random_state=123)

    # Splitting train, validate, and test dfs on x and y
    x_train = train.drop(columns=['home_value'])
    x_validate = validate.drop(columns=['home_value'])
    x_test = test.drop(columns=['home_value'])

    y_train = train['home_value']
    y_validate = validate['home_value']
    y_test = test['home_value']
    
    return x_train, y_train, x_validate, y_validate, x_test, y_test


def prep_zillow(df):
    """
    Handles nulls
    optimizes or fixes data types
    handles outliers w/ manual logic
    clean variables via dropping columns and renaming features
    includes feature engineering 
    returns a clean dataframe
    Splits df into train, validate, test, and associated dfs on x and y 
    """

    df = drop_bad_columns(df)

    df = drop_nulls(df)

    df = handle_nulls(df)

    df = optimize_types(df)

    df = handle_outliers(df)

    df = clean_variables(df)

    df = feature_engineering(df)

    train, validate, test = zillow_split_explore(df)

    x_train, y_train, x_validate, y_validate, x_test, y_test = zillow_split_model(df)

    # df.to_csv("zillow.csv", index=False)

    return df, train, validate, test, x_train, y_train, x_validate, y_validate, x_test, y_test



In [ ]:
train, validate, test = zillow_split_explore(df)

In [ ]:
train.head()

In [ ]:
#Summary statistics
train.describe().T

In [ ]:
# plotly 3d scatter
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.update_layout(width=700, height=500)
# fig.update_layoubt(margin={"r":0,"t":0,"l":10,"b":0})
fig.show()